In [ ]:
"""See markdown"""
# pylint: disable=line-too-long, redefined-outer-name, import-error

Analyze prediction values of correct vs false predictions. Can we find a good prediction score threshold that lets us eliminate important errors?

In [ ]:
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go

In [ ]:
path = (
    Path.home()
    / "downloads"
    / "temp"
    / "sex3_full-10fold-validation_prediction_augmented-all.csv"
)
df = pd.read_csv(path, index_col=0, header=0)

In [ ]:
classes = df["True class"].unique()

In [ ]:
df["harmonized_donor_sex"].value_counts()

In [ ]:
for label in classes:
    df_label = df[df["True class"] == label]
    fig = go.Figure()

    # Iterate classes each target and add a violin plot for it
    for target in classes:
        vals = df_label[df_label["Predicted class"] == target]["Max pred"]
        print(df_label["assay_epiclass"].value_counts())

        fig.add_trace(
            go.Violin(
                y=vals,
                name=f"{target} ({len(vals)})",
                box_visible=True,
                meanline_visible=True,
                points="all",
            )
        )

    fig.update_layout(
        title_text=f"Predicted value distribution for {label} ({df_label.shape[0]})",
        yaxis_title="Prediction score",
        xaxis_title="Target",
    )
    fig.update_yaxes(range=[1 / 3, 1.01])

    fig.show()

In [ ]:
for label in classes:
    df_label = df[df["True class"] == label][df["assay_epiclass"] == "input"]
    fig = go.Figure()

    # Iterate classes each target and add a violin plot for it
    for target in classes:
        vals = df_label[df_label["Predicted class"] == target]["Max pred"]

        fig.add_trace(
            go.Violin(
                y=vals,
                name=f"{target} ({len(vals)})",
                box_visible=True,
                meanline_visible=True,
                points="all",
            )
        )

    fig.update_layout(
        title_text=f"Predicted value distribution for {label} ({df_label.shape[0]})",
        yaxis_title="Prediction score",
        xaxis_title="Target",
    )
    fig.update_yaxes(range=[1 / 3, 1.01])

    fig.show()